In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dtale
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import r2_score

%matplotlib
%matplotlib inline
pd.options.display.float_format = '{:.4f}'.format #set it to convert scientific noations such as 4.225108e+11 to 422510842796.00
pd.set_option('display.max_columns', 100) #  display all the columns
pd.set_option('display.max_rows', 100) # display all the rows
np.set_printoptions(suppress=True,formatter={'float_kind':'{:f}'.format})

Using matplotlib backend: MacOSX


# Loading data

In [143]:
df = pd.read_csv('Data/X_train_prepared.csv')
df = df.sample(10000)
df.drop('Unnamed: 0',axis='columns',inplace=True)
X_train = df.drop('loan_status',axis='columns')
y_train = df[['loan_status']]

In [144]:
df = pd.read_csv('Data/X_test_prepared.csv')
df = df.sample(1000)
df.drop('Unnamed: 0',axis='columns',inplace=True)
X_test = df.drop('loan_status',axis='columns')
y_test = df[['loan_status']]

# Label encoding for y_train

In [145]:
le = preprocessing.LabelEncoder().fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

# Random Forest 

In [146]:
model = RandomForestClassifier(max_depth=20, random_state=0, n_estimators=100)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [147]:
predictions = model.predict(X_test)

In [148]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

In [153]:
print('Still need to fix the get_dummies problem !!!!')
print('*******')
print('Preliminary results for Random Forest:')
print('train score: ',train_score)
print('test score: ',test_score)

Still need to fix the get_dummies problem !!!!
*******
Preliminary results for Random Forest:
train score:  0.9999
test score:  0.937


# K-nearest neighbours

In [158]:
k_nearest = KNeighborsClassifier(n_neighbors=5)
k_nearest.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [159]:
train_score = k_nearest.score(X_train, y_train)
test_score = k_nearest.score(X_test, y_test)

In [163]:
print('Preliminary results for K-nearest neighbours:')
print('train score: ',train_score)
print('test score: ',test_score)

Preliminary results for K-nearest neighbours:
train score:  0.8494
test score:  0.783


# !!!!! USING GRIDSEARCH !!!!!

In [69]:
param_grid = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1400, 1600, 1800, 2000]}

rfc = RandomForestRegressor(n_jobs=-1,oob_score = True, verbose=2)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_)
best_model = CV_rfc.best_estimator_